In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-question-answering-dataset/train-v1.1.json
/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json


In [3]:
import numpy as np
import pandas as pd
import transformers
import json
import warnings
from datasets import Dataset
warnings.filterwarnings("ignore")

In [4]:
with open('/kaggle/input/stanford-question-answering-dataset/train-v1.1.json') as train_file:
    train_data = json.load(train_file)

In [5]:
with open('/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json') as dev_file:
    dev_data = json.load(dev_file)

In [6]:
def prepare_dataset(data):
    contexts = []
    questions = []
    answers = []

    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]  # Take the first answer
                answer['text'] = answer['text']
                answer['answer_start'] = answer['answer_start']

                contexts.append(context)
                questions.append(question)
                answers.append(answer)
    
    return Dataset.from_dict({'context': contexts, 'question': questions, 'answers': answers})

In [7]:
train_dataset = prepare_dataset(train_data)
dev_dataset = prepare_dataset(dev_data)

In [8]:
from transformers import AutoTokenizer

model_name='distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples['question']]
    inputs = tokenizer(
        questions,
        examples['context'],
        max_length=384,
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True,  # Now supported by the fast tokenizer
        return_tensors="pt"
    )
    
    start_positions = []
    end_positions = []
    
    for i, answer in enumerate(examples['answers']):
        start_positions.append(answer['answer_start'])
        end_positions.append(answer['answer_start'] + len(answer['text']))
    
    inputs.update({
        "start_positions": start_positions,
        "end_positions": end_positions,
    })
    
    return inputs

In [10]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_dev_dataset = dev_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_dev_dataset
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,5.270300,5.141836
2,4.956100,4.948071


TrainOutput(global_step=5476, training_loss=5.260709908863678, metrics={'train_runtime': 3606.132, 'train_samples_per_second': 48.583, 'train_steps_per_second': 1.519, 'total_flos': 1.7167621364554752e+16, 'train_loss': 5.260709908863678, 'epoch': 2.0})

In [13]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer,device=0)

In [20]:
context="Snehee studies in Mumbai."
question='What does Snehee study?'

result = qa_pipeline({
    'context': context,
    'question': question
})

In [21]:
print("Prediction:", result)

Prediction: {'score': 0.0381229966878891, 'start': 18, 'end': 24, 'answer': 'Mumbai'}


In [22]:
predicted_start = result['start']
predicted_end = result['end']


true_answer = "Mumbai"
true_start = context.find(true_answer)
true_end = true_start + len(true_answer)

In [23]:
def compute_iou(pred, ref):
    pred_tokens = set(range(pred['start_positions'], pred['end_positions']))
    ref_tokens = set(range(ref['start_positions'], ref['end_positions']))
    intersection = len(pred_tokens & ref_tokens)
    union = len(pred_tokens | ref_tokens)
    return intersection / union if union != 0 else 0

In [24]:
pred = {'start_positions': predicted_start, 'end_positions': predicted_end}
ref = {'start_positions': true_start, 'end_positions': true_end}

# Compute token-level IoU
iou_score = compute_iou(pred, ref)

In [25]:
print("Prediction:", result['answer'])
print("True Answer:", true_answer)
print("Token-level IoU:", iou_score)

Prediction: Mumbai
True Answer: Mumbai
Token-level IoU: 1.0
